In [1]:
#imports 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import spacy

In [2]:
pd.__version__

'1.5.3'

In [9]:
# pip install spacy

Note: you may need to restart the kernel to use updated packages.Collecting spacy
     --------------------------------------- 12.6/12.6 MB 16.4 MB/s eta 0:00:00



  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached murmurhash-1.0.9-cp38-cp38-win_amd64.whl (18 kB)
  Using cached cymem-2.0.7-cp38-cp38-win_amd64.whl (30 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
     ---------------------------------------- 2.2/2.2 MB 14.1 MB/s eta 0:00:00
  Using cached srsly-2.4.6-cp38-cp38-win_amd64.whl (482 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached preshed-3.0.8-cp38-cp38-win_amd64.whl (96 kB)
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
  Using cached thinc-8.1.9-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached blis-0.7.9-cp38-cp38-win_amd64.whl (7.0 MB)
  Using cached confection-0.0.4-py3-none-any.whl (32 kB)
  Using cached colorama-0.4.6-py2.py3-none


You should consider upgrading via the 'C:\Users\20193635\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [5]:
#loading data
Email = pd.read_csv('email_headers.csv', encoding='cp1252')
Employee = pd.read_excel('EmployeeRecords.xlsx')

Email = pd.read_csv('email_headers.csv', encoding='cp1252')
df_text = Email[['From', 'Subject']].drop_duplicates('Subject')
df_text['Subject'].to_csv('subject.csv', index=False)
# df_text = pd.read_excel('Subject.xlsx')
df_text['Subject'] = df_text['Subject'].apply(lambda x: x[4:] if x.startswith('Re: ') or x.startswith('RE: ') else x )
df_text = df_text.drop_duplicates(subset = ['Subject'], keep='first' )
nlp=spacy.load("en_core_web_lg")
df_text['token'] = df_text['Subject'].apply(lambda x: nlp(x))
df_text


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# !pip uninstall gensim

In [ ]:
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags, STOPWORDS


In [ ]:
df_class = pd.DataFrame(columns = ['Class', 'Description'], data= [['Spam', 'Spam'],['Social', 'Anniversary Birthday Event Social Night Evening'],['Work', 'Report Team Data Meeting Inspection Visit IPO Protocol Supplies GasTech Files'], [ 'Change/schedule', 'Change Schedule Appointments Meeting Deadline New Shift Announcement'],['Weird', 'Weird Plants text and drive'],['Other', 'Other No Hurry'],['Undefined', 'Undefined']])
df_class['token'] = df_class['Description'].apply(lambda x: nlp(x))
df_class

classes = df_class['Class']
for index, row in df_class.iterrows():
    row
    x = df_text['token'].apply(lambda x: x.similarity(row['token']))
    print(row['Class'])


def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=4, max_len=15) #originally len 2 and 15


df_text['tagged_docs'] = df_text.apply(lambda row: TaggedDocument(tokenize(row['Subject']), [str(row.name)]), axis=1)


df_text['c_spam'] = df_text.apply(lambda x: 0 if x['class_spam']> 0.5 else -1, axis =1)
df_text['c_social'] = df_text.apply(lambda x: 1 if x['class_social']> 0.5 else -1, axis =1)
df_text['c_work'] = df_text.apply(lambda x: 2 if x['class_work']> 0.5 else -1, axis =1)
df_text['c_schedule'] = df_text.apply(lambda x: 3 if x['class_schedule']> 0.5 else -1, axis =1)
df_text['c_weird'] = df_text.apply(lambda x: 4 if x['class_weird']> 0.5 else -1, axis =1)
df_text['c_undefined'] = df_text.apply(lambda x: 5 if x['class_undefined']> 0.5 else -1, axis =1)

df_text['max'] = df_text[['class_spam', 'class_social', 'class_work', 'class_schedule', 'class_weird', 'class_other', 'class_undefined']].idxmax(axis=1)

df_text['C'] = -2
for index, row in df_text.iterrows():
    cl = []
    # row.
    if row['class_spam']>0.5:
        cl.append(0)
    if row['class_social']>0.5: 
        cl.append(1)
    if row['class_work']>0.5: 
        cl.append(2)
    if row['class_schedule']>0.5: 
        cl.append(3)
    if row['class_weird']>0.5: 
        cl.append(4)
    if row['class_other']>0.5: 
        cl.append(5)
    if row['class_undefined']>0.5: 
        cl.append(6)
    else:
        cl.append(-1)
    print(index)
    df_text.loc[index,'C'] = (cl)


In [ ]:
df_text.to_csv('email_classification.csv')

In [ ]:
# LargeEmail = LargeEmail.merge(df_text, on='Subject')